In [1]:
import cvxpy as cp
import numpy as np
from foldLine import data,x,y,z
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
from conowlLocal import neighborhood
from kMeans import kMeans, min_k_index, max_k_index
from copy import deepcopy

In [2]:
n_sample=len(data[0])
n_feature=len(data)

In [3]:
K=100
epsilon=1

In [4]:
indexs=np.random.choice(range(n_feature),K)
B=np.zeros((n_feature,n_feature))
I=[]
for i in range(n_feature):
    I.append(i)

In [5]:
def owl(beta):
    z = 0.001 * cp.norm1(beta) #lambda
    for i in range(1,90): #r
        z = z+cp.sum_largest(beta, i)*0.5 #delta
    return z

def loss_fn(X, Y, beta):
    return cp.norm2(cp.matmul(X, beta) - Y)**2

def regularizer(beta):
    return cp.norm1(beta)

def objective_fn(X, Y, beta, lambd):
    return loss_fn(X, Y, beta) + lambd * regularizer(beta)

def mse(X, Y, beta):
    return (1.0 / X.shape[0]) * loss_fn(X, Y, beta).value

def objective_f1(X, Y, beta):
    return loss_fn(X, Y, beta) + owl(beta)

In [6]:
for index in indexs:
    kIndex = neighborhood(data[index],data,epsilon)
    n1 = len(kIndex)
    X = np.zeros((n_sample,n1))
    for i in range(n1):
        X[:,i]=data[kIndex[i]]
    Y = np.array(data[index])
    #c = np.ones(n1)
    beta = cp.Variable(n1)
    #constraints = [c*beta==1]
    problem = cp.Problem(cp.Minimize(objective_f1(X, Y, beta)))
    problem.solve()
    coef = beta.value
    for i in range(n1):
        B[kIndex[i],index]=coef[i]

TypeError: 'NoneType' object is not subscriptable

In [ ]:
"""
beta = cp.Variable(n)
lambd = cp.Parameter(nonneg=True)
constraints = [c*beta==1]
problem = cp.Problem(cp.Minimize(objective_fn(X_train, Y_train, beta, lambd)))

lambd_values = np.logspace(-2, 3, 50)
train_errors = []
test_errors = []
beta_values = []
for v in lambd_values:
    lambd.value = v
    problem.solve()
    train_errors.append(mse(X_train, Y_train, beta))
    test_errors.append(mse(X_test, Y_test, beta))
    beta_values.append(beta.value)
"""

In [ ]:
nonzero=[]
for index in range(n_feature):
    nonzero.append(np.count_nonzero(B[:,index]))
nonzero_idx=[]
for i in range(n_feature):
    if nonzero[i]!=0:
        nonzero_idx.append(i)

max(nonzero)
nonzero

In [ ]:
"""
for i in range(len(nonzero_idx)):
    index=nonzero_idx[i]
    n=nonzero[index]
    idx=[]
    for i in range(1,n+1):
        idx.append(max_k_index(B[:,index],i))
    ax=plt.subplot(111)
    ax.scatter(y,z,color='yellow')
    ax.scatter(y[index],z[index],color='red')
    for i in range(n):
        ax.scatter(y[idx[i]],z[idx[i]],color='black')
    plt.show()
"""

In [ ]:
for i in range(len(nonzero_idx)):
    plt.vlines(400,min(B[:,nonzero_idx[i]]),max(B[:,nonzero_idx[i]]),color='c',linestyles='dashed')
    plt.vlines(1400,min(B[:,nonzero_idx[i]]),max(B[:,nonzero_idx[i]]),color='c',linestyles='dashed')
    plt.scatter(range(1800),B[:,nonzero_idx[i]])
    plt.scatter(nonzero_idx[i],0,color='red')
    plt.show()

In [ ]:
for i in range(len(nonzero_idx)):
    idx=[]
    for j in range(1,9):
        idx.append(max_k_index(B[:,nonzero_idx[i]],j))
    colour=['midnightblue','blue','mediumslateblue','dodgerblue','deepskyblue','lightskyblue','powderblue','lightblue']
    ax=plt.subplot(111)
    ax.scatter(y,z,color='yellow')
    ax.scatter(y[nonzero_idx[i]],z[nonzero_idx[i]],color='red')
    for j in range(8):
        ax.scatter(y[idx[j]],z[idx[j]],color=colour[j])
    plt.show()

In [ ]:
index=nonzero_idx[12]
idx=[]
for j in range(1,9):
    idx.append(max_k_index(B[:,index],j))
colour=['midnightblue','blue','mediumslateblue','dodgerblue','deepskyblue','lightskyblue','powderblue','lightblue']
ax=plt.subplot(111)
ax.scatter(y,z,color='yellow')
ax.scatter(y[index],z[index],color='red')
for i in range(8):
    ax.scatter(y[idx[i]],z[idx[i]],color=colour[i])

In [ ]:
sorted(B[:,nonzero_idx[5]],reverse=True)

In [ ]:
for index in indexs:
    kIndex = deepcopy(I)
    kIndex.remove(index)
    n1 = len(kIndex)
    X = np.zeros((n_sample,n1))
    for i in range(n1):
        X[:,i]=data[kIndex[i]]
    Y = np.array(data[index])
    c = np.ones(n1)
    beta = cp.Variable(n1)
    #lambd = cp.Parameter(nonneg=True)
    #t = cp.Parameter(nonneg=True)
    constraints = [c*beta==1]
    problem = cp.Problem(cp.Minimize(objective_f1(X, Y, beta)),constraints)
    #lambd.value = 0.1
    #t.value=0.15
    problem.solve()
    coef = beta.value
    for i in range(n1):
        B[kIndex[i],index]=coef[i]